# Trabajando con series temporales

Pandas se desarrolló con objetivo del modelado financiero, por lo que, como era de esperar, contiene un conjunto bastante extenso de herramientas para trabajar con fechas, horas y datos indexados por tiempo.
Los datos de fecha y hora vienen tienen más de una representación:

- **Marcas temporales** hacen referencia a momentos particulares en el tiempo (por ejemplo, July 4th, 2015 at 7:00am).
- **Intervalos de tiempo** y **periodos** hacen referencia a un período de tiempo entre dos puntos determinados; por ejemplo, el año 2015. Los *periods* generalmente hacen referencia a un caso especial de intervalos de tiempo en los que cada intervalo tiene una duración uniforme y no se superpone (por ejemplo, períodos de 24 horas que comprenden días).
- **Deltas temporales** o **duraciones** hacen referencia a un período de tiempo exacto (por ejemplo, una duración de 22.56 segundos).

En esta sección, veremos cómo trabajar con cada uno de estos tipos de datos de fecha/hora en Pandas.

Esto no será una guía completa de las herramientas de series de tiempo disponibles en Python o Pandas, sino mñas bien una descripción general de cómo podemos abordar el trabajo con series temporales.

Comenzaremos con las herramientas para lidiar con fechas y horas en Python, antes de pasar más específicamente a las herramientas proporcionadas por Pandas.

Después de enumerar algunos recursos más profundos, veremos algunos ejemplos breves de cómo trabajar con datos temporales en Pandas.

## Fechas y horas en Pandas

El mundo de Python tiene varias representaciones disponibles de fechas, horas, deltas e intervalos de tiempo.
Si bien las herramientas de series temporales proporcionadas por Pandas tienden a ser las más útiles para las aplicaciones de ciencia de datos, es útil ver su relación con otros paquetes utilizados en Python.

### Fechas y horas nativas de Python: ``datetime`` y ``dateutil``

Los objetos básicos de Python para trabajar con fechas y horas residen en el módulo built-in ``datetime``.
Junto con el módulo de terceros ``dateutil``, podemos usarlos para realizar fácilmente una serie de funciones muy útiles con fechas y horas.
Por ejemplo, podemos crear manualmente una fecha usando el tipo ``datetime``:

In [2]:
from datetime import datetime
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

O, usando el módulo ``dateutil``, puedes convertir a fecha desde strings con varios formatos:

In [3]:
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

Pero puede inferir muchos más:

In [4]:
print(parser.parse("2015/10/12"))
print(parser.parse("2015-10-12"))
# Ojo, por defecto, en este formato interpreta mes-día-año
print(parser.parse("12-10-2015"))

2015-10-12 00:00:00
2015-10-12 00:00:00
2015-12-10 00:00:00


Una vez que tenemos el objeto ``datetime``, podemos hacer cosas como imprimir el día de la semana:

In [5]:
date.strftime('%A')

'Saturday'

### EJERCICIO

Interpreta las siguientes fechas y obtén qué día de la semana fue o va a ser (algunas las tendrás que interpretar tú y otras directamente):
1. "2020-09-15"
2. "12th October, 1492"
3. 20 de Enero de 1999
4. 7 de Marzo de 2077
5. "1512/02/01"
6. "2021-05-22"

En la última línea, hemos usado uno de los códigos de formato de cadena estándar para imprimir fechas (``"% A"``), sobre el cual podemos leer en la [sección de strftime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) de la [documentación de Python](https://docs.python.org/3/library/datetime.html).

La documentación de otras utilidades de fecha se puede encontrar en la [documentación en línea de dateutil](http://labix.org/python-dateutil).
Un paquete relacionado a tener en cuenta es [``pytz``](http://pytz.sourceforge.net/), que contiene herramientas para trabajar con zonas horarias.

El poder de ``datetime`` y ``dateutil`` radica en su flexibilidad y sintaxis sencilla: podemos usar estos objetos y sus métodos integrados para realizar fácilmente casi cualquier operación que nos pueda interesar.
Donde fallan es cuando queremos trabajar con grandes conjuntos de fechas y horas:
Así como las listas de variables numéricas de Python son subóptimas en comparación con los arrays numéricos de NumPy, las listas de objetos de fecha y hora de Python son subóptimas en comparación con los arrays tipados de fechas.

### Arrays tipados de fechas: ``datetime64`` de Numpy

Las debilidades del formato de fecha y hora de Python inspiraron al equipo de NumPy a agregar un conjunto de tipos de datos de series temporales nativos a su propia librería.
El dtype ``datetime64`` codifica las fechas como enteros de 64 bits y, por lo tanto, permite que losa rrays de fechas se representen de manera muy compacta.
El ``datetime64`` requiere un formato de entrada muy específico:

In [7]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

Sin embargo, una vez que tengamos esta fecha formateada, podemos realizar rápidamente operaciones vectorizadas con ella:

In [8]:
date + np.arange(12) # suma días

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

Gracias al tipado uniforme en los arrays de NumPy ``datetime64``, este tipo de operación se puede realizar mucho más rápido que si estuviéramos trabajando directamente con los objetos ``datetime`` de Python, especialmente a medida que los arrays se hacen más grandes.

Un detalle de los objetos ``datetime64`` y ``timedelta64`` es que están construidos en una unidad de tiempo fundamental.
Dado que el objeto ``datetime64`` está limitado a una precisión de 64 bits, el rango de tiempos codificables es $2^{64}$ veces esta unidad fundamental.
En otras palabras, ``datetime64`` impone un trade-off entre la resolución de tiempo y el período de tiempo máximo.

Por ejemplo, si deseas una resolución de tiempo de un nanosegundo, solo tiene suficiente información para codificar un rango de $2^{64}$ nanosegundos, que es algo menos de 600 años.
NumPy inferirá la unidad deseada a partir de la entrada; por ejemplo, aquí hay una fecha a nivel de día:

In [9]:
np.datetime64('2015-07-04') # guarda hasta el día

numpy.datetime64('2015-07-04')

Y aquí otra a nivel de minutos:

In [10]:
np.datetime64('2015-07-04 12:00') # guarda hasta los minutos

numpy.datetime64('2015-07-04T12:00')

Observa que la zona horaria se establece automáticamente como la hora local del ordenador que ejecuta el código.
Puedes forzar cualquier unidad fundamental utilizando uno de los muchos códigos de formato; por ejemplo, aquí forzaremos un tiempo basado en nanosegundos:

In [11]:
np.datetime64('2015-07-04 12:59:59.50', 'ns') # fuerzo que se guarde en nanosegundos

# la precisión de la variable datetime64 depende de hasta dónde se guarde el detalle de la fecha
# a más detalle, menos periodo de tiempo abarcado

numpy.datetime64('2015-07-04T12:59:59.500000000')

La siguiente tabla, extraída de la [docuemntación de datetime64 de NumPy](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html), enumera los códigos de formato disponibles junto con los períodos de tiempo relativos y absolutos que puede codificar:

|Code    | Meaning     | Time span (relative) | Time span (absolute)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

Para los tipos de datos que vemos en el mundo real, el valor predeterminado es ``datetime64[ns]``, ya que puede codificar un rango útil de fechas con una precisión adecuada.

Finalmente, notaremos que si bien el tipo de datos ``datetime64`` aborda algunas de las deficiencias del tipo incorporado de Python ``datetime``, carece de muchos de los métodos y funciones convenientes proporcionados por ``datetime`` y ``dateutil``.
Puede encontrar más información en la [documentación del datetime64 de NumPy](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html).

### EJERCICIO

Interpreta las siguientes fechas como ``datetime64`` según el periodo relativo que mejor se adapte:
1. "2020-09-15 00:00"
2. 12th October, 1492 (al nanosegundo)
3. 20 de Enero de 1999 a las 15:24:10
4. 7 de Marzo de 2077 01:01:01.00000001
5. "1512/02/01" a las 23:00
6. "1512/02/01" a las 23:30:10.00000034
7. "1512/02/01" a las 23:30:10.00000034 como segundos
8. "2021-05-22" como microsegundos


¿Has observado algo raro? ¿Entiendes por qué pasa?

### Fechas y horas en Pandas: lo mejor de ambos mundos

Pandas se basa en todas las herramientas que acabamos de comentar para proporcionar un objeto ``Timestamp``, que combina la facilidad de uso de ``datetime`` y ``dateutil`` con el almacenamiento eficiente y la interfaz vectorizada de ``numpy.datetime64``.

A partir de un grupo de estos objetos de ``Timestamp``, Pandas puede construir un ``DatetimeIndex``, que se puede usar para indexar datos en un ``Series`` o ``DataFrame``; veremos muchos ejemplos de esto a continuación.

Por ejemplo, podemos usar las herramientas de Pandas para repetir la demostración de arriba.
Podemos analizar una fecha de cadena con formato flexible y usar códigos de formato para generar el día de la semana:

In [13]:
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

In [14]:
date.strftime('%A')

'Saturday'

Adicionalmente, podemos hacer operaciones al estilo de NumPy con estos objetos:

In [15]:
date + pd.to_timedelta(np.arange(12), 'D') # suma 12 días

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

También podemos utilizar diferentes frecuencias, no solo diario, tal como indica esta tabla, aunque para esta función no podremos utilizar todas ellas:

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

### EJERCICIO

Ayudándote de la tabla anterior, crea una función que tome como parámetro:
 - fecha: un string con el formato de fecha "YYYY-MM-DD" (año-mes-día)
 - valor: un entero con el que se hará una operación sobre la fecha recibida
 - unidad: un string que valdrá "microsegundos", "milisegundos", "segundos", "minutos", "horas", "días" o "semanas"
 - operación": sring que sea "+" o "-"
 
La dunción deberá recoger un string con formato de fecha al que se le sumará o restará "valor" "unidad", en función del parámetro "operación". Habrá que traducir el campo "unidad" a frecuencias entendibles por la función de Pandas.

### EJERCICIO

Haz las iguientes operaciones y devuelve el día de la semana que fue el último de esa fecha:

1. "2020-09-15" + 15 días
2. "12th October, 1492" - 100 milisegundos
3. 20 de Enero de 1999 a las 15:24:10 + 2 minutos
4. 7 de Marzo de 2077 01:01:01.00000001 - 1 año
5. "1512/02/01" a las 23:00 + 5 nanosegundos
6. "1512/02/01" a las 23:30:10 -7 meses
7. "2021-05-22" - 24 días
8. "1984-10-01" - 370 semanas


En la siguiente sección, veremos más de cerca la manipulación de datos de series de tiempo con las herramientas proporcionadas por Pandas.

## Series temporales de Pandas: Indexando por tiempo

Donde las herramientas de series temporales de Pandas se vuelven realmente útiles es cuando comenzamos a indexar datos por marcas temporales.
Por ejemplo, podemos construir un objeto ``Series`` que tenga datos indexados por tiempo del siguiente modo:

In [18]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

Ahora que tenemos estos datos en una ``Series``, podemos hacer uso de cualquiera de los patrones de indexación de las ``Series`` que discutimos en las secciones anteriores, pasando valores que se puedan convertir en fechas:

In [19]:
data['2014-07-04':'2015-07-04']

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

Hay operaciones adicionales de indexación especiales exclusivas de las fechas, como pasar un año para obtener una porción de todos los datos de ese año:

In [20]:
data['2015']

2015-07-04    2
2015-08-04    3
dtype: int64

Más adelante, veremos ejemplos adicionales de la conveniencia de las fechas como índices.
Pero antes, es interesante ver las estructuras de datos de series de tiempo disponibles.

## Estructuras de datos de series temporales en Pandas

A continuación, presentaremos las estructuras de datos fundamentales de Pandas para trabajar con datos de series temporales:

- Para *marcas temporales*, Pandas proporciona el tipo ``Timestamp``. Como se mencionó anteriormente, es esencialmente un reemplazo del ``datetime`` nativo de Python, pero se basa en el tipo de datos ``numpy.datetime64`` más eficiente. La estructura de índice asociada es `` DatetimeIndex ''.
- Para *periodos de tiempo*, Pandas proporciona el tipo ``Period``. Esto codifica un intervalo de frecuencia fija basado en ``numpy.datetime64``. La estructura de índice asociada es ``PeriodIndex``.
- Para *deltas de tiempo* o *duraciones*, Pandas proporciona el tipo ``Timedelta``. ``Timedelta`` es un reemplazo más eficiente para el tipo nativo de Python ``datetime.timedelta`` y se basa en ``numpy.timedelta64``. La estructura de índice asociada es ``TimedeltaIndex``.

Lo más importante de estos objetos de fecha/hora son los objetos ``Timestamp`` y ``DatetimeIndex``.
Si bien estos objetos se pueden invocar directamente instanciándolos desde su clase, es más común usar la función ``pd.to_datetime()``, que puede analizar una amplia variedad de formatos e inferirlos automáticamente.

Pasar una sola fecha a ``pd.to_datetime()`` devuelve un ``Timestamp``; pasar una serie de fechas por defecto devuelve un ``DatetimeIndex``:

In [26]:
from datetime import datetime 
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

# año mes día

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

Cualquier ``DatetimeIndex`` puede ser convertido a ``PeriodIndex`` con la función ``to_period()`` añadiendo el código de la frecuencia. En este caso, usaremos ``'D'`` para indicar frecuencia diaria:

In [29]:
dates.to_period('D')

PeriodIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
             '2015-07-08'],
            dtype='period[D]', freq='D')

Fíjate que, si tenemos fechas diarias, utilizar una frecuencia no acorde como puede ser mensual, produce resultados diferentes:

In [31]:
dates.to_period('M')

PeriodIndex(['2015-07', '2015-07', '2015-07', '2015-07', '2015-07'], dtype='period[M]', freq='M')

Los ``TimedeltaIndex`` aparecen, por ejemplo, cuando se restan dos fechas:

In [32]:
dates - dates[0]

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

### EJERCICIOS

Ahora que ya hemos aprendido un poco más de esto, vamos a hacer ejercicios que realmente sirvan para algo:
1. Lee el dataset de reseñas de yelp! (../../data/yelp_academic_dataset_review.csv)
2. Quédate solo con las columnas "stars" y "date"
3. Convierte la columna "date" en formato fecha
4. Quédate con 1 registro por día y haz la media de stars
5. Cuál es la mayor diferencia de días sin registros?
6. Vuelve al df original y crea un DatetimeIndex a partir de la columna 'date'. (Puede que no valga directamente con usar el ``Series``, sino que haya que tener los datos en cierto tipo de variable, mira el ejemplo)
7. Créate, a partir de esa variable DatetimeIndex, una nueva variable PeriodIndex con frecuencia mensual, y asígnala a una nueva columna "date_M"
8. Agrupa por esta variable y obtén el máximo, el mínimo y el total de votaciones por mes. ¿Qué mes ha sido el que más reseñas ha recibido?